# Webscraping Reddit
## Opinion Mining & Sentiment Analysis in Product Subreddits

### Packages used:
* PRAW (Python Reddit API Wrapper)
* VADER (Valence Aware Dictionary and sEntiment Reasoner)
* nltk

### Links used:
* [PRAW tutorial](https://towardsdatascience.com/scraping-reddit-with-praw-76efc1d1e1d9)
* [PRAW doc](https://praw.readthedocs.io/en/latest/index.html)
* [VADER github](https://github.com/cjhutto/vaderSentiment)

In [3]:
import praw
import pandas as pd

from datetime import datetime as dt

In [4]:
# Create the instance of praw. All credetials stored in
# praw.ini file 
reddit = praw.Reddit()

print(reddit.user.me())


ClientException: Required configuration setting 'client_id' missing. 
This setting can be provided in a praw.ini file, as a keyword argument to the `Reddit` class constructor, or as an environment variable.

In [ ]:
# create instances for the subreddits to investigate
dtvnow = reddit.subreddit('DirecTVNow')

In [ ]:
# use top posts of the last year.
dtv_subs = dtvnow.controversial(time_filter = 'year', limit = 1000)
# dtv_subs = dtvnow.top(time_filter = 'year', limit = 1000)

In [ ]:
# With PRAW, everything is saved into a submission datatype
# containing: title, score (upvotes), dateTime, author, etc
# lets collect all of that
dtvdata = {
    "title"        : [],
    "score"        : [],
    "datetime"     : [],
    "author"       : [],
    "num_comments" : [],
    "selftext"     : [],
    "permalink"    : []
}

In [ ]:
for submission in dtv_subs:
    dtvdata["title"].append(submission.title)
    dtvdata["score"].append(submission.score)
    dtvdata["datetime"].append(dt.utcfromtimestamp(submission.created_utc))
    dtvdata["author"].append(submission.author)
    dtvdata["num_comments"].append(submission.num_comments)
    dtvdata["selftext"].append(submission.selftext)
    dtvdata["permalink"].append("https://www.reddit.com" + submission.permalink)

In [ ]:
# export as a pandas df

dtv_df = pd.DataFrame(dtvdata)

In [ ]:
# dtv_df
# move to a csv
dtv_df.to_csv("./dtvnow.csv")

In [ ]:
# Display some of the results we got
dtv_df.head(5)


In [ ]:
# We now can work with VADER to do sentiment analysis on each post and title
# because subreddits dont always contain text body, we want to concat the title
# and selftext together to analyze all text per post.
title_text_zip = zip(dtv_df.title, dtv_df.selftext)
title_text = []
for ti, txt in title_text_zip:
    title_text.append(ti + " "+ txt)

# title_text

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# import vader analyzer. This will conduct the sentiment analysis
vder = SentimentIntensityAnalyzer()
resp = []
for text in title_text:
    analysis = vder.polarity_scores(text)
    resp.append(analysis)

for i in range(10):
    print(resp[i])

In [ ]:
# we have 4 identifiers that are returned:
# neg(ative), neu(tral), pos(itive), and compound.
# compound will produce the net sentiment of a post.
# pos if compound >=.05, neg if compound <=-.05, neu othw

# add both sentiment and compound score to df
vader_anlys = {
    "sentiment" : [],
    "compound"  : [],
    "pos_cnt"   : 0,
    "neg_cnt"   : 0,
    "neu_cnt"   : 0
}

for i in resp:
    if i['compound']<= -0.05:
        vader_anlys["sentiment"].append("negative") 
        vader_anlys["compound"].append(i['compound'])
        vader_anlys["neg_cnt"] += 1
    elif i['compound']>= 0.05:
        vader_anlys["sentiment"].append("positive") 
        vader_anlys["compound"].append(i['compound'])
        vader_anlys["pos_cnt"] += 1
    else:
        vader_anlys["sentiment"].append("neutral") 
        vader_anlys["compound"].append(i['compound'])
        vader_anlys["neu_cnt"] += 1
        
        
dtv_df["sentiment"] = vader_anlys["sentiment"]
dtv_df["compound"] = vader_anlys["compound"]


In [ ]:
dtv_df.head(10)

In [ ]:
# export data to a csv

dtv_df.to_csv("./dtvnow_with_sentiment.csv")

In [ ]:
print(dtv_df['sentiment'].value_counts())

fig, ax = plt.subplots()
dtv_df['sentiment'].value_counts().plot(ax=ax, kind='pie', autopct='%1.0f%%')

In [ ]:
# Let's now take a look at the positives and negative
# posts and see if we can find any specific key words

# we can use nltk for this
import collections
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
# from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import nltk

# DataFrames containing just pos/neg sentiment posts
neg_df = dtv_df.loc[dtv_df['sentiment'] == 'negative']
pos_df = dtv_df.loc[dtv_df['sentiment'] == 'positive']

# def to concat title and text
def title_text(zip_gen):
    zip_arr = []
    for ti, tx in zip_gen:
        zip_arr.append(ti + " " + tx)
    
    return zip_arr

neg_df_zip = zip(neg_df.title, neg_df.selftext)
pos_df_zip = zip(pos_df.title, pos_df.selftext)

neg_txt = title_text(neg_df_zip)
pos_txt = title_text(pos_df_zip)

print(neg_txt[0])

In [ ]:
# Use nltk to remove stop words to increase frequency clarity
# put into fcn
def remove_stop_wds(text_list):
    # regex found here: https://gist.github.com/ameyavilankar/10347201
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words('english'))
    for i in range(len(text_list)):
        text = text_list[i].lower()
        word_tokens = tokenizer.tokenize(text)
        filtered = [w for w in word_tokens if not w in stop_words]
        text_list[i] = filtered
    
remove_stop_wds(neg_txt)
remove_stop_wds(pos_txt)

print(neg_txt[0])

In [ ]:
# we can create a function that displays the top 10 
# most common words, and output to a pandas df
def word_freq_df(text_list):
    frq = {}
    for txt in text_list:
        word_frequency = nltk.FreqDist(txt)
        for word, frequency in word_frequency.most_common():
            if word not in frq:
                frq[word] = frequency
            else:
                frq[word] += frequency
    w_f = {
        'word'      : [],
        'frequency' : []
    }
    for w in frq.keys():
        w_f['word'].append(w)
        w_f['frequency'].append(frq[w])
    
    frequency_df = pd.DataFrame(w_f).sort_values(by=['frequency'], ascending = False)
    return frequency_df
    
neg_df = word_freq_df(neg_txt)
pos_df = word_freq_df(pos_txt)
# neg_df
# pos_df.head(100)

ngx = neg_df[:20].plot.bar(x='word',y='frequency', legend=False, rot = 50)
ngx.set_ylabel('frequency')
psx = pos_df[:20].plot.bar(x='word',y='frequency', legend=False, rot = 50)
psx.set_ylabel('frequency')

In [ ]:
dtv_df.loc[dtv_df['sentiment'] == 'negative'].to_csv("./dtvnow_negative_sentiment.csv")
dtv_df.loc[dtv_df['sentiment'] == 'positive'].to_csv("./dtvnow_positive_sentiment.csv")